### Callback API Task

In [1]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print("train dataset shape:", train_images.shape, train_labels.shape)
print("test dataset shape:", test_images.shape, test_labels.shape)

train dataset shape: (60000, 28, 28) (60000,)
test dataset shape: (10000, 28, 28) (10000,)


In [2]:
from tensorflow.keras.layers import Layer,Dense,Flatten,Input
from tensorflow.keras.models import Model


INPUT_SIZE = 28

def create_model():
    input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE))
    x = Flatten()(input_tensor)
    x = Dense(64, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    return model

In [3]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets

def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets)
    oh_targets = to_categorical(targets)

    return images, oh_targets

def get_train_valid_test(train_images, train_targets, test_images, test_targets, validation_size=0.2, random_state=124):
    train_images, train_oh_targets = get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)

    train_images, validation_images, train_oh_targets, validation_oh_targets = \
    train_test_split(train_images, train_oh_targets, stratify=train_oh_targets, test_size=validation_size, random_state=random_state)

    return (train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets)

In [4]:
from tensorflow.keras.datasets import mnist

(train_images, train_targets), (test_images, test_targets) = mnist.load_data()

(train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets) = \
get_train_valid_test(train_images, train_targets, test_images, test_targets)

print(train_images.shape, train_oh_targets.shape)
print(validation_images.shape, validation_oh_targets.shape)
print(test_images.shape, test_oh_targets.shape)

(48000, 28, 28) (48000, 10)
(12000, 28, 28) (12000, 10)
(10000, 28, 28) (10000, 10)


In [25]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

mcp_cb = ModelCheckpoint(
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

# mcp_cb = ModelCheckpoint(
#     filepath="./callback_files/model.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.model.keras",
#     monitor='val_loss',
#     # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
#     save_best_only=False,
#     save_weights_only=False,
#     mode='min'
# )

history = model.fit(x=train_images, 
                    y=train_oh_targets, 
                    validation_data=(validation_images, validation_oh_targets), 
                    batch_size=64, 
                    epochs=20, 
                    callbacks=[mcp_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - acc: 0.8329 - loss: 0.5895 - val_acc: 0.9459 - val_loss: 0.1855
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 870us/step - acc: 0.9519 - loss: 0.1617 - val_acc: 0.9546 - val_loss: 0.1473
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 878us/step - acc: 0.9677 - loss: 0.1070 - val_acc: 0.9665 - val_loss: 0.1092
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 896us/step - acc: 0.9754 - loss: 0.0830 - val_acc: 0.9712 - val_loss: 0.0991
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 891us/step - acc: 0.9812 - loss: 0.0647 - val_acc: 0.9751 - val_loss: 0.0864
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 922us/step - acc: 0.9845 - loss: 0.0518 - val_acc: 0.9722 - val_loss: 0.0950
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 871us/step - acc: 0.9868 - loss: 0.0429 - val_acc: 0.9728 - val_loss: 0.0901
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 930us/step - acc: 0.9888 - loss: 0.0362 - val_acc: 0.9760 - val_loss: 0.0840
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s

In [7]:
!dir callback_files

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: DADA-845F

 C:\KDT_231228_khj\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-28  오전 10:58    <DIR>          .
2024-05-28  오전 10:57    <DIR>          ..
2024-05-28  오전 10:57           743,334 model.001-0.1817-0.9029.model.keras
2024-05-28  오전 10:57           743,334 model.002-0.1244-0.9559.model.keras
2024-05-28  오전 10:57           743,334 model.003-0.1135-0.9685.model.keras
2024-05-28  오전 10:57           743,334 model.004-0.0973-0.9747.model.keras
2024-05-28  오전 10:57           743,334 model.005-0.0977-0.9798.model.keras
2024-05-28  오전 10:57           743,334 model.006-0.0912-0.9823.model.keras
2024-05-28  오전 10:57           743,334 model.007-0.1068-0.9849.model.keras
2024-05-28  오전 10:57           743,334 model.008-0.0936-0.9872.model.keras
2024-05-28  오전 10:57           743,334 model.009-0.1140-0.9900.model.keras
2024-05-28  오전 10:57           743,334 model.010-0.1046-0.9906.model.keras
2024-05-28  오전 10:57           743,334 model.011-0.

In [26]:
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - acc: 0.9717 - loss: 0.1438


[0.1287546455860138, 0.9750999808311462]

In [27]:
model = create_model()
model.load_weights('./callback_files/weights.008-0.0840-0.9875.weights.h5')

model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - acc: 0.9689 - loss: 0.0964


[0.08809623122215271, 0.9729999899864197]

In [28]:
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 451us/step - acc: 0.9689 - loss: 0.0964


[0.08809623122215271, 0.9729999899864197]

In [22]:
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - acc: 0.9694 - loss: 0.1559


[0.13407786190509796, 0.973800003528595]

In [23]:
from tensorflow.keras.models import load_model

model = load_model('./callback_files/model.008-0.0906-0.9866.model.keras')
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step - acc: 0.9713 - loss: 0.1040


[0.09351637214422226, 0.974399983882904]

In [24]:
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step - acc: 0.9713 - loss: 0.1040


[0.09351637214422226, 0.974399983882904]

In [29]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

history = model.fit(x=train_images, 
                    y=train_oh_targets, 
                    validation_data=(validation_images, validation_oh_targets), 
                    batch_size=64, 
                    epochs=20, 
                    callbacks=[rlr_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8186 - loss: 0.6133 - val_acc: 0.9493 - val_loss: 0.1750 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 860us/step - acc: 0.9570 - loss: 0.1476 - val_acc: 0.9602 - val_loss: 0.1267 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 861us/step - acc: 0.9707 - loss: 0.0984 - val_acc: 0.9641 - val_loss: 0.1168 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 850us/step - acc: 0.9767 - loss: 0.0740 - val_acc: 0.9704 - val_loss: 0.0941 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 862us/step - acc: 0.9822 - loss: 0.0605 - val_acc: 0.9712 - val_loss: 0.0940 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step - acc: 0.9842 - loss: 0.0515 - val_acc: 0.9714 - val_loss: 0.0940 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 825us/step - acc: 0.9867 - loss: 0.0424 - val_acc: 0.9676 - val_loss: 0.1099 - learning_rate: 0

In [30]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=2,
    mode='min'
)

history = model.fit(x=train_images, 
                    y=train_oh_targets, 
                    validation_data=(validation_images, validation_oh_targets), 
                    batch_size=64, 
                    epochs=20, 
                    callbacks=[ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8253 - loss: 0.5982 - val_acc: 0.9463 - val_loss: 0.1817
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 886us/step - acc: 0.9532 - loss: 0.1586 - val_acc: 0.9506 - val_loss: 0.1561
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 876us/step - acc: 0.9663 - loss: 0.1120 - val_acc: 0.9666 - val_loss: 0.1108
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 873us/step - acc: 0.9766 - loss: 0.0779 - val_acc: 0.9662 - val_loss: 0.1080
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 880us/step - acc: 0.9810 - loss: 0.0637 - val_acc: 0.9688 - val_loss: 0.0962
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 842us/step - acc: 0.9845 - loss: 0.0529 - val_acc: 0.9633 - val_loss: 0.1216
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 836us/step - acc: 0.9866 - loss: 0.0439 - val_acc: 0.9699 - val_loss: 0.0979


In [32]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

mcp_cb = ModelCheckpoint(
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=2,
    mode='min'
)

history = model.fit(x=train_images, 
                    y=train_oh_targets, 
                    validation_data=(validation_images, validation_oh_targets), 
                    batch_size=64, 
                    epochs=20, 
                    callbacks=[mcp_cb, rlr_cb, ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8280 - loss: 0.6067 - val_acc: 0.9444 - val_loss: 0.1835 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 946us/step - acc: 0.9540 - loss: 0.1584 - val_acc: 0.9604 - val_loss: 0.1343 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 951us/step - acc: 0.9675 - loss: 0.1102 - val_acc: 0.9674 - val_loss: 0.1076 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 991us/step - acc: 0.9770 - loss: 0.0785 - val_acc: 0.9683 - val_loss: 0.1044 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 886us/step - acc: 0.9802 - loss: 0.0627 - val_acc: 0.9681 - val_loss: 0.1027 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 926us/step - acc: 0.9846 - loss: 0.0524 - val_acc: 0.9680 - val_loss: 0.1049 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 894us/step - acc: 0.9851 - loss: 0.0458 - val_acc: 0.9742 - val_loss: 0.0895 - learning_rate: 0